In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB

from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion




In [30]:
data_dir = os.path.expanduser("~/Documents/financial-news-dataset/ReutersNews106521")

def parse_article(file_name):
    with open(file_name, 'r') as f:
        try:
            news = f.readlines()
        except:
            print(file_name)
            raise NotImplementedError()

    news = news[4:]  # removes the dataset header
    news = " ".join(news).replace("\n", "")
    return news
    
#print(parse_article(data_dir + "/20090106/" + "us-alcoa-idUSTRE5056SB20090106"))

In [33]:

limit = 1000
def load_articles(limit):
    news_data = []
    counter = 0
    for directory in os.listdir(data_dir):
        os.chdir(data_dir + "/" + directory)
        for news_article in os.listdir(data_dir + "/" + directory):
            counter += 1
            news_data.append(parse_article(news_article))
            if counter >= limit:
                break
        else:
           continue
        break #allows the inner break to break the outer loop
    return news_data


In [34]:
news_data = load_articles(1000)
print(len(news_data))

1000


In [76]:
keywords = ["oil", "exxon", "WTI", "crude", "OPEC", "BP", "Rosneft", "drill", "barrel", "price"]

keywords = [keyword.lower() for keyword in keywords]

def check_title(news_article, keywords, min_matches=2):
    """
    Checks to see if the article title has atleast as many
    keyword matches as min_matches to determine if it's
    relevant to the topic of oil
    """
    with open(news_article, 'r') as f:
        title = f.readline()
    title = title.lower()
    return len(list(set(title.split(" ")) & set(keywords))) >= min_matches
    

print(check_title(data_dir + "/20090106/" + "us-alcoa-idUSTRE5056SB20090106",
                  keywords))


False


In [79]:
def make_labels(keywords, limit=1000, min_matches=1):
    labels = []
    counter = 0
    for directory in os.listdir(data_dir):
        os.chdir(data_dir + "/" + directory)
        for news_article in os.listdir(data_dir + "/" + directory):
            counter += 1
            labels.append(check_title(news_article, keywords, min_matches))
            if counter >= limit:
                break
        else:
           continue
        break #allows the inner break to break the outer loop
    return labels

labels = make_labels(keywords, limit = 100000)
print(np.sum(labels))
labels[0:10]

[False, False, False, False, False, False, False, False, False, False]

3672


In [80]:
print(np.sum(labels) / len(labels))

0.03672


so about 3.7% of the articles are relevant to us out of the first 100,000.  We'll play with this a little more, but let's see about fitting some classification models.

In [82]:
lim = 100000
news_data = load_articles(lim)
labels = make_labels(keywords, limit = lim)

In [83]:
X_train, y_train, X_test, y_test = train_test_split(news_data,
                                                    labels,
                                                    test_size = 0.33,
                                                    random_state = 322)

In [84]:
len(X_train)

67000

In [85]:
cv = CountVectorizer()

model = LogisticRegression()

pipe = Pipeline([("vect", cv), ("model", model)])



In [86]:
pipe.fit(X_train, y_train)


MemoryError: 

In [ ]:
preds = pipe.predict(X_test)
acc = np.mean(preds == y_test)